In [3]:
# Braxton Morrow 
#SNHU
#Summer 2022

from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
from dash_table.Format import Format, Group, Scheme, Symbol
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

# changed animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from Animal_Shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "accuser"
password = "Password97"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({})

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('My Dashboard')

image_filename = 'Project_2_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div(
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard Braxton Morrow'))),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children = [        
# Adds in code for the interactive filtering options. 
    dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DIT'},
                {'label': 'Reset', 'value':'R'}
            ],
            value='R',
            labelStyle={'display': 'inline-block'}
            ),
         html.A([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),style={
                    'height' : '60%',
                    'width' : '60%',
                    'float' : 'right',
                    'position' : 'relative',
                    'padding-top' : 1,
                    'padding-right' : 1
                })],
                href='www.snhu.edu')              
    ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[{'name': 'Animal ID', 'id': 'animal_id', 'type': 'text', 'editable': False},
            {'name': 'Type', 'id': 'animal_type', 'type': 'text'},
            {'name': 'Breed', 'id': 'breed', 'type': 'text'},
            {'name': 'Color', 'id': 'color', 'type': 'text'},
            {'name': 'Birthday', 'id': 'date_of_birth', 'type': 'date'},
            {'name': 'DateTime', 'id': 'datetime', 'type': 'datetime'},
            {'name': 'monthyear', 'id': 'monthyear', 'type': 'datetime'},
            {'name': 'Animal Name', 'id': 'name', 'type': 'text'},
            {'name': 'Outcome', 'id': 'outcome_type', 'type': 'text'},
            {'name': 'Outcome Subtype', 'id': 'outcome_subtype', 'type': 'text'},
            {'name': 'Sex', 'id': 'sex_upon_outcome', 'type': 'text'},
            {'name': 'Age', 'id': 'age_upon_outcome_in_weeks', 'type': 'numeric',
             'format': Format(
                scheme=Scheme.fixed, 
                precision=2,
                decimal_delimiter='.')},
            {'name': 'Latitude', 'id': 'location_lat', 'type':'numeric',
            'format': Format(
                scheme=Scheme.fixed, 
                precision=3,
                group=Group.yes,
                groups=3,
                group_delimiter=' ',
                decimal_delimiter='.')},
            {'name': 'Longitude', 'id': 'location_long', 'type': 'numeric',
            'format': Format(
                scheme=Scheme.fixed, 
                precision=3,
                group=Group.yes,
                groups=3,
                group_delimiter=' ',
                decimal_delimiter='.')}
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
# Sets up the features for interactive data table to make it user-friendly for client
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
        style_as_list_view=True,
    style_cell={'padding': '5px'},
    style_header={
        'backgroundColor': 'grey',
        'fontWeight': 'bold'
    },
    style_cell_conditional=[
        {
            'if': {'column_id': c},
            'textAlign': 'left'
        } for c in ['Date', 'Region']
    ]
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that the chart and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
# Adds code to filter interactive data table with MongoDB queries
        {'name': 'Animal ID', 'id': 'animal_id', 'type': 'text', 'editable': False},
        {'name': 'Type', 'id': 'animal_type', 'type': 'text'},
        {'name': 'Breed', 'id': 'breed', 'type': 'text'},
        {'name': 'Color', 'id': 'color', 'type': 'text'},
        {'name': 'Birthday', 'id': 'date_of_birth', 'type': 'date'},
        {'name': 'DateTime', 'id': 'datetime', 'type': 'datetime'},
        {'name': 'monthyear', 'id': 'monthyear', 'type': 'datetime'},
        {'name': 'Animal Name', 'id': 'name', 'type': 'text'},
        {'name': 'Outcome', 'id': 'outcome_type', 'type': 'text'},
        {'name': 'Outcome Subtype', 'id': 'outcome_subtype', 'type': 'text'},
        {'name': 'Sex', 'id': 'sex_upon_outcome', 'type': 'text'},
        {'name': 'Age', 'id': 'age_upon_outcome_in_weeks', 'type': 'numeric',
             'format': Format(
                scheme=Scheme.fixed, 
                precision=2,
                decimal_delimiter='.')},
        {'name': 'Latitude', 'id': 'location_lat', 'type':'numeric',
            'format': Format(
                scheme=Scheme.fixed, 
                precision=3,
                group=Group.yes,
                groups=3,
                group_delimiter=' ',
                decimal_delimiter='.')},
        {'name': 'Longitude', 'id': 'location_long', 'type': 'numeric',
            'format': Format(
                scheme=Scheme.fixed, 
                precision=3,
                group=Group.yes,
                groups=3,
                group_delimiter=' ',
                decimal_delimiter='.')}  
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        return (data,columns)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#BFEEFF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    # convert data to pandas dataframe
        dff = pd.DataFrame.from_dict(viewData)
        # count by times breed occurs
        new_df = dff.breed.value_counts()
        new_df.name = "breed"
        new_df = new_df.to_frame()
        new_df.reset_index(inplace=True)
        
        # return updated graph data
        return [
        dcc.Graph(
        figure = px.pie(new_df, values="breed", names="index")
        )
        ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
# Adds in the code for your geolocation chart
odff = df if viewData is None else pd.DataFrame.from_dict(viewData)
        if row is None:
            dff = odff
        else:
            dff = odff.iloc[row,:]

        map = [
            dl.Map(style={'width': '1000px', 'height': '550px'}, center=[30,-97], zoom=10, children = [
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                    dl.Tooltip(dff.iloc[0,4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[0,9])
                    ])
                ])
            ])
        ]
        return map


app

SyntaxError: invalid syntax (1941618672.py, line 39)